# Cleaning the csv dataset

objective: create a dataframe where there is 1 gtd_id, 1 headline, 1 link

Cleaning the 2_GTD_case_details_2015_2017.csv dataset into id,link

let us have 2 dataframe s_df and df2

# Link collecting

In [1]:
import pandas as pd
import asyncio
import aiohttp
import ssl

from bs4 import BeautifulSoup
from collections import Counter
import re
from urllib.parse import urlparse
import time
import ast


import requests

from IPython.display import display


#conda install tornado==4.5.3 as tornado 5x is not good for this

#asynschronously fetch one url
async def fetch(session, url):
    async with session.get(url, ssl=ssl.SSLContext()) as response:
        return await response.text()

#asynschronously fetch multiple    
async def fetch_all(urls, loop):
    async with aiohttp.ClientSession(loop=loop) as session:
        results = await asyncio.gather(*[fetch(session, url) for url in urls], return_exceptions=True)
        return results

#function to return a soup object
def soup(page):
    try:
        soup = BeautifulSoup(page, 'html.parser')
        return soup
    except:
        soup = []
        return soup

#function to return a list of texts in divs
def div_soup(page):
    try:
        soup = BeautifulSoup(page, 'html.parser')
        tags = soup.find_all('div')
        tags_list = [t.text for t in tags]
        return tags_list
    except:
        tags_list = []
        return tags_list    
    
#function to return a list of soup objects
def soup_list(urls):
    loop = asyncio.new_event_loop()
    htmls = loop.run_until_complete(fetch_all(urls, loop))
    soup_list = list(map(soup,htmls))
    loop.close()
    return soup_list

df=pd.read_csv('0.1_1_google_link_4.csv').dropna().drop(['Unnamed: 0'],axis=1).reset_index(drop=True)
df_links = df.drop_duplicates(subset=['link'])
df_links.head()

,headline,link,GTD ID:
0,4 civilians killed in ambush by suspected BIFF...,https://newsinfo.inquirer.net/699492/police-of...,201506180097
1,"Trooper killed, 2 wounded in Davao Del Sur cla...",https://en.wikipedia.org/wiki/Battle_of_Marawi,201506190018
2,"Trooper killed, 2 wounded in Davao Del Sur cla...",https://www.google.com.sg/search?source=hp&ei=...,201506190018
3,"Trooper killed, 2 wounded in Davao Del Sur cla...",https://en.wikipedia.org/wiki/Timeline_of_the_...,201506190018
4,"Trooper killed, 2 wounded in Davao Del Sur cla...",https://www.crisisgroup.org/crisiswatch/print?...,201506190018


In [2]:
url_list = [i for i in df_links['link']]
id_list = [i for i in df_links['GTD ID:']]
display(url_list,id_list)

['https://newsinfo.inquirer.net/699492/police-officer-slashed-suspect-shot-dead-outside-nyc-subway',
 'https://en.wikipedia.org/wiki/Battle_of_Marawi',
 'https://www.google.com.sg/search?source=hp&ei=bD2jXNb8JMH2vAShgYX4BA&q=%22Trooper+killed%2C+2+wounded+in+Davao+Del+Sur+clash+with+rebels%22+Philippines+News+Agency+June+20%2C+2015.&oq=%22Trooper+killed%2C+2+wounded+in+Davao+Del+Sur+clash+with+rebels%22+Philippines+News+Agency+June+20%2C+2015.&gs_l=psy-ab.3...395.865..894...0.0..0.0.0.......0....1..gws-wiz.....0.#',
 'https://en.wikipedia.org/wiki/Timeline_of_the_CPP%E2%80%93NPA%E2%80%93NDF_rebellion',
 'https://www.crisisgroup.org/crisiswatch/print?page=1&location%5B0%5D=46&t=CrisisWatch+Database+Filter',
 'http://www.refworld.org/pdfid/4d5a310e2.pdf',
 'https://www.asienhaus.de/12all/index.php?action=social&c=09c6c3783b4a70054da74f2538ed47c6.1379',
 'https://www.asienhaus.de/archiv/user_upload/AMP_-_Human_Rights_Report_Philippines_2017_final.pdf',
 'https://www.hrw.org/report/2009/04

[201506180097,
 201506190018,
 201506190018,
 201506190018,
 201506190018,
 201506190018,
 201506190018,
 201506190018,
 201506190018,
 201506190018,
 201506190018,
 201506190018,
 201506190033,
 201506190033,
 201506190033,
 201506190033,
 201506190033,
 201506190033,
 201506190033,
 201506190033,
 201506190033,
 201506190033,
 201506200017,
 201506200017,
 201506200017,
 201506200017,
 201506200017,
 201506200017,
 201506200017,
 201506200017,
 201506200017,
 201506200017,
 201506210058,
 201506210058,
 201506210058,
 201506210058,
 201506210058,
 201506210058,
 201506210058,
 201506210058,
 201506210058,
 201506220045,
 201506220045,
 201506220045,
 201506220045,
 201506220045,
 201506220045,
 201506220045,
 201506220045,
 201506220045,
 201506220094,
 201506220094,
 201506220094,
 201506230030,
 201506230030,
 201506230030,
 201506230030,
 201506230030,
 201506230030,
 201506230030,
 201506230030,
 201506230030,
 201506230030,
 201506240031,
 201506240031,
 201506250018,
 201506280

execution

In [8]:
total = 13594
a = 13590
b = 13594
counter = a

#this code runs through the df_links list, returns a h5 file with 1 website and all its divs in text form.
while b <= 13594:
    loop = asyncio.new_event_loop()
    print('Getting ' +str(a) + ' to ' +str(b))
    urls = url_list[a:b]
    htmls = loop.run_until_complete(fetch_all(urls, loop))
    #print(htmls[-1])
    soups = list(map(div_soup,htmls))
    lst = list(zip(url_list[a:b],id_list[a:b],soups))
    l_df = pd.DataFrame(lst)
    #df2 = pd.concat([df2,l_df])
    #print(type(l_df[2][0]))
    
    for i in range(len(l_df)):
        gtd_id = l_df[1][i]
        l_df.iloc[i].to_pickle('{}_{}.h5'.format(gtd_id,counter))
        print('{}_{}.h5'.format(gtd_id,counter))
        counter +=1

    a += 5
    b += 5

    loop.close()


Getting 13590 to 13594
201506160033_13590.h5
201506160034_13591.h5
201506160034_13592.h5
201506160034_13593.h5


In [10]:
pd.read_pickle('201506160034_13592.h5')


0    https://www.rappler.com/nation/79266-npa-kill-...
1                                         201506160034
2    [\n, \n, \n HELLO FEATURE MARKUP VAL: \n\nWelc...
Name: 2, dtype: object